Zadanie 2

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [2]:
#Wczytuję dane
plik = pd.read_csv('/content/drive/MyDrive/Biologia systemów/single-cell-tracks_exp1-6_noErbB2.csv.gz')
metadata = pd.read_csv('/content/drive/MyDrive/Biologia systemów/01-readme-experiment-description_2022-04-05.csv')

In [3]:
print(plik.columns)
# /\ wypisuje nazwy wszystkich kolumn

print(metadata.columns)

Index(['Exp_ID', 'Image_Metadata_Site', 'track_id', 'Image_Metadata_T',
       'Nuclear_size', 'ERKKTR_ratio', 'FoxO3A_ratio',
       'objNuclei_Location_Center_X', 'objNuclei_Location_Center_Y'],
      dtype='object')
Index(['Site', 'Cell_Line', 'Mutation', 'Biosensors', 'Conditions',
       'Acquisition_frequency_min', 'Duration_h'],
      dtype='object')


In [4]:
#Tworzę słownik mutacji z pliku metadata
site_to_mutation = dict(zip(metadata['Site'], metadata['Mutation']))

# Mapuję słownik do danych z pliku tworząc nową kolumnę z typem mutacji
plik['Mutation'] = plik['Image_Metadata_Site'].map(site_to_mutation)

In [5]:
print(plik.columns)
# /\ wypisuje nazwy wszystkich kolumn

print(metadata.columns)

Index(['Exp_ID', 'Image_Metadata_Site', 'track_id', 'Image_Metadata_T',
       'Nuclear_size', 'ERKKTR_ratio', 'FoxO3A_ratio',
       'objNuclei_Location_Center_X', 'objNuclei_Location_Center_Y',
       'Mutation'],
      dtype='object')
Index(['Site', 'Cell_Line', 'Mutation', 'Biosensors', 'Conditions',
       'Acquisition_frequency_min', 'Duration_h'],
      dtype='object')


Baseline - wyznaczam go jako 10% dolnych wartości w przebiegu. Aktywne komórki muszą go przekroczyć o co najmniej jedno odhylenie standardowe

In [6]:
pd.options.mode.string_storage = 'python'
import numpy as np

# Wybieram ścieżkę sygnałową
signal_col = 'ERKKTR_ratio'

# Obliczam baseline dla każdej komórki
def define_baseline(group):
    baseline = group[signal_col].quantile(0.1)
    std = group[signal_col].std()
    group['is_active'] = group[signal_col] > (baseline + std)
    return group

pogrupowany_plik = plik.groupby('track_id').apply(define_baseline)
print('zrobione')

zrobione


<ipython-input-6-75be8b3e07c4>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pogrupowany_plik = plik.groupby('track_id').apply(define_baseline)


In [7]:
print(pogrupowany_plik.columns)

Index(['Exp_ID', 'Image_Metadata_Site', 'track_id', 'Image_Metadata_T',
       'Nuclear_size', 'ERKKTR_ratio', 'FoxO3A_ratio',
       'objNuclei_Location_Center_X', 'objNuclei_Location_Center_Y',
       'Mutation', 'is_active'],
      dtype='object')


In [ ]:
from scipy.spatial import KDTree

# Ustaw promień sąsiedztwa w mikrometrach (możesz dostosować)
radius = 50

# Lista do wyników
results = []

# Iterujemy po każdej klatce czasowej i pozycji (miejsce na płytce)
for (time, site), frame in pogrupowany_plik.groupby(['Image_Metadata_T', 'Image_Metadata_Site']):
    coords = frame[['objNuclei_Location_Center_X', 'objNuclei_Location_Center_Y']].values
    tree = KDTree(coords)

    # Wektor aktywności
    is_active = frame['is_active'].values.astype(bool)

    # Dla każdej komórki znajdujemy indeksy sąsiadów w zadanym promieniu
    neighbors = tree.query_ball_point(coords, r=radius)

    # Liczymy aktywnych sąsiadów
    active_neighbor_counts = [
        is_active[neigh].sum() - is_active[i]  # odejmujemy siebie samego jeśli aktywny
        for i, neigh in enumerate(neighbors)
    ]

    # Dodajemy do wyniku
    frame_result = frame[['track_id', 'Image_Metadata_T', 'Mutation']].copy()
    frame_result['active_neighbors'] = active_neighbor_counts
    frame_result['is_active'] = is_active
    results.append(frame_result)

# Łączymy wyniki
df_neighbors = pd.concat(results, ignore_index=True)
